As described in: Lin, M.-S., et al.: Malicious URL filtering- a big data application. IEEE Interna-
tional Conference on Big Data (2013) we are going to extract features from URLs

In [53]:
import re
from urllib.parse import urlparse
from sklearn import *
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import joblib

Step 1: Lexical features extraction: split URLs into components, apply a sliding window to the domain, and use a bag-of-words model to describe each component.

In [54]:
def lexicalFE(url): #lexical feature extraction fn - takes in a URL
    # Ensure the URL has a scheme for proper parsing, crucial for IPv6 addr
    if not urlparse(url).scheme:
        url = 'http://' + url  # prepend with default scheme
    
    try:
        parsed_url = urlparse(url)
        domain = parsed_url.netloc
        path = parsed_url.path
        query = parsed_url.query
        domain = domain.replace('www.', '')
        features = {
            'domain': domain,
            'domain_length': len(domain),
            'path_length': len(path),
            'query_length': len(query),
            'num_path_components': len(path.split('/')) - 1,  # Subtracting 1 because the leading '/' results in an empty string at the start
            'num_query_components': len(query.split('&')) if query else 0,  # Only count if there's a query
        }
        features['has_digits_in_domain'] = any(char.isdigit() for char in domain)
        return features
    except ValueError as e: #handle errors TODO: more here
        print(f"Error processing URL {url}: {e}")
        return {}

Step 2: descriptive features extraction - this function will further split the path component, remove common prefixes and TLDs, and calc stats

In [55]:
def descriptiveFE(url): #descriptive feature extraction fn - takes in a URL
    parsed_url = urlparse(url)
    domain = parsed_url.netloc.replace('www.', '')  # Remove common prefix
    path = parsed_url.path
    query = parsed_url.query
    path_components = path.split('/') # further split the path
    filename = path_components[-1] if '.' in path_components[-1] else None
    file_extension = filename.split('.')[-1] if filename else None
    
    # Calculate statistics
    features = {
        'domain_length': len(domain),
        'path_length': len(path),
        'query_length': len(query),
        'num_path_components': len(path_components),
        'filename': filename,
        'file_extension': file_extension,
        'is_ip_address': bool(re.match(r'^\d{1,3}(\.\d{1,3}){3}$', domain)),
        'executable_extension': file_extension in ['exe', 'bin', 'bat']
    }
    return features

Step 3: integrate and test

In [56]:
#CAUTION: neutered phishing URL: urls = ['{http colon slash slash}clt1658125{dot}benchurl{dot}com']
urls = ['https://www.example.org/bin.exe?arg=value', 'http://blog.example.com:443/executable.exe?arg=test123']

for url in urls:
    print(f"URL: {url}")
    lexFeats = lexicalFE(url) # lexical features
    descFeats = descriptiveFE(url) # descriptive features
    print("Lexical Features:", lexFeats)
    print("Descriptive Features:", descFeats)
    print("\n")

URL: https://www.example.org/bin.exe?arg=value
Lexical Features: {'domain': 'example.org', 'domain_length': 11, 'path_length': 8, 'query_length': 9, 'num_path_components': 1, 'num_query_components': 1, 'has_digits_in_domain': False}
Descriptive Features: {'domain_length': 11, 'path_length': 8, 'query_length': 9, 'num_path_components': 2, 'filename': 'bin.exe', 'file_extension': 'exe', 'is_ip_address': False, 'executable_extension': True}


URL: http://blog.example.com:443/executable.exe?arg=test123
Lexical Features: {'domain': 'blog.example.com:443', 'domain_length': 20, 'path_length': 15, 'query_length': 11, 'num_path_components': 1, 'num_query_components': 1, 'has_digits_in_domain': True}
Descriptive Features: {'domain_length': 20, 'path_length': 15, 'query_length': 11, 'num_path_components': 2, 'filename': 'executable.exe', 'file_extension': 'exe', 'is_ip_address': False, 'executable_extension': True}




In [57]:
df = pd.read_csv('malicious_phish-kaggle-thishusseinali.csv', names=['URL', 'Classification'])

In [58]:
# Apply lexical and descriptive feature extraction
df['Lexical_Features'] = df['URL'].apply(lambda x: lexicalFE(x))
df['Descriptive_Features'] = df['URL'].apply(lambda x: descriptiveFE(x))

testIndex = 2
# Print the first row as an example (you can change the index to print another row)
print(df.iloc[testIndex])

Error processing URL http://RybjUxÙãl5»7ÆE%ÝÔk+h|U+ýk©ìÉ½Æq]âF·õÁ¢w)ëA·ç°{t*m!¦2: Invalid IPv6 URL
Error processing URL http://ÆeF§÷%¶¿Õ½9¿b@Ö¸ÚZE¤ÒC¢ÄÅª2åç-]W³fU¤Jgkz.ø¿nJçåæuøD%@ðûÇùM¹uË: Invalid IPv6 URL
Error processing URL http://Ó6¸RTÃu~æÙg0>÷mÖiÓ=;XZ\%êýÜÉfn&\°%7õÉ"ieÖ1ÄÁêFÐò<$cï6t[0ò2"/Æa^2âpù/ýãÇ$E¬R«È²ú[Ì¶p¥qÒ°i°^ò[»³»]±9êdÓS¿Ë]ùþ5j¿·ªocÂplà7ÊÏJ§¢#3ðDCDõ²çÇGÝ.Vò=¿QB§Ä'`ÊáZÉê ÔîÆm®ÍÝQÓ(z;¹Áê¬âytÖÙ®ëNP²ÜEQ: Invalid IPv6 URL
Error processing URL http://µÔA¨!ÝÛ=]º£¦Pôwr72-ÕY5Äòè7¬-³]×)&¡e¸¢À6RD­NvY¨Ð«Ñ3Â¸%Qñ+ÛÈ¸$¶gz{þ: Invalid IPv6 URL
Error processing URL http://¨RÊÃûaCóÞit×ßÂe-DÖØ+9YèÌçÏ¯·"0£ÙÕ.0ößF«7¹NRÙ{ccÉÄãéçx[Ä6a5Ñ³LÖíÜÉÀ£Òma¥yRX*0ÅÝ7×ÊÁÌo«Õs¶0kdèÑ&Ä"Ï¨mZ'àDM×ñXÚÒK"päî±h¬cAÊeK@4r"^'ÓFþ1*ËË PÞô;õ$úàÑ@þ=êWÑ"Ãhñ®ç^«Ýó^çRúUJ.<6CyÜFØrÿV2ôæýZãiiIb;¨Ëµu^ÍVy)­è»âýº+SÖáÃì?å6åÔ/: Invalid IPv6 URL
Error processing URL ht

In [59]:
# Select the row index you're interested in
row_index = testIndex  # For example, to print the second row

# Convert the dictionary to a JSON string for pretty printing
lexical_features_str = json.dumps(df.at[row_index, 'Lexical_Features'], indent=4)
descriptive_features_str = json.dumps(df.at[row_index, 'Descriptive_Features'], indent=4)

# Print the features
print(f"Lexical Features for row {row_index}:\n{lexical_features_str}\n")
print(f"Descriptive Features for row {row_index}:\n{descriptive_features_str}\n")

Lexical Features for row 2:
{
    "domain": "mp3raid.com",
    "domain_length": 11,
    "path_length": 24,
    "query_length": 0,
    "num_path_components": 2,
    "num_query_components": 0,
    "has_digits_in_domain": true
}

Descriptive Features for row 2:
{
    "domain_length": 0,
    "path_length": 35,
    "query_length": 0,
    "num_path_components": 3,
    "filename": "krizz_kaliko.html",
    "file_extension": "html",
    "is_ip_address": false,
    "executable_extension": false
}



Now, we need to normalize the features and concatenate them with the original dataframe

In [60]:
chunk_size = 5000 

In [61]:
# instantiate empty dataframe
df_final = pd.DataFrame()

# process in chunk size defined in previous cell
for start in range(0, df.shape[0], chunk_size):
    end = min(start + chunk_size, df.shape[0])
    df_chunk = df.iloc[start:end].copy()
    df_chunk.reset_index(drop=True, inplace=True)
    # normalize lexical and descriptive features
    lexFeatsDF = pd.json_normalize(df_chunk['Lexical_Features'])
    lexFeatsDF.columns = ['Lexical_' + str(col) for col in lexFeatsDF.columns]
    descFeatsDF = pd.json_normalize(df_chunk['Descriptive_Features'])
    descFeatsDF.columns = ['Descriptive_' + str(col) for col in descFeatsDF.columns]
    df_chunk = pd.concat([df_chunk, lexFeatsDF, descFeatsDF], axis=1) #concat normalized feats with chunk
    df_final = pd.concat([df_final, df_chunk], axis=0, ignore_index=True) #direct append to final df

# drop unnecessary column names
df_final.drop(['Lexical_Features', 'Descriptive_Features'], axis=1, inplace=True)

In [62]:
df_final = df_final.drop(1) # drop row 1
df_final = df_final.drop(['URL'], axis=1)
df_final_columns = df_final.columns.tolist()
with open('model_columns.txt', 'w') as f:
    f.write('\n'.join(df_final_columns))
df_final = df_final.dropna() #drop rows with missing values
catCols = df_final.select_dtypes(include=['object', 'category']).columns
# convert categoricals
for col in catCols:
    # Skip the target column 'Classification'
    if col == 'Classification':
        continue
    le = LabelEncoder()
    df_final[col] = le.fit_transform(df_final[col])
    # save the encoder
    joblib.dump(le, 'categorical_feature_encoder.joblib')

# split dataframe into features and target
X = df_final.drop('Classification', axis=1)
y = df_final['Classification']

# Convert 'Classification' to numerical values if it's categorical
if y.dtype == 'object' or y.dtype.name == 'category':
    le = LabelEncoder()
    y = le.fit_transform(y)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a decision tree classifier
dTree = DecisionTreeClassifier(random_state=42)
dTree.fit(X_train, y_train)

print(f"class mapping: {dict(zip(le.classes_, le.transform(le.classes_)))}")

class mapping: {'benign': 0, 'defacement': 1, 'malware': 2, 'phishing': 3}


In [63]:
# predict test set results
y_pred = dTree.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the Decision Tree model: {accuracy:.2f}")

Accuracy of the Decision Tree model: 0.94


In [64]:
# encoder saved from before - training phase
encoder = joblib.load('categorical_feature_encoder.joblib')

def predict_url_classification(url, dTree, df_final_columns):
    # Extract features
    lexical_features = lexicalFE(url)
    descriptive_features = descriptiveFE(url)
    
    # Combine features
    all_features = {**lexical_features, **descriptive_features}
    
    # Create a DataFrame for the features
    features_df = pd.DataFrame([all_features])
    
    # Ensure the DataFrame matches the training data structure
    # Add missing columns with default values
    for col in df_final_columns:
        if col not in features_df.columns:
            features_df[col] = 0  # Or another appropriate default value
    
    # Reorder columns to match the training data
    features_df = features_df[df_final_columns]
    
    # Drop columns that are not features (e.g., 'URL', 'Classification' if they were included)
    features_to_drop = ['URL', 'Classification']  # Adjust based on your actual data
    features_df = features_df.drop(columns=[col for col in features_to_drop if col in features_df.columns], errors='ignore')
    
    # Predict the classification
    prediction = dTree.predict(features_df)
    
    return prediction[0]  # Assuming binary classification for simplicity

# Use with caution if you are pasting in real malicious domains
# CAUTION # url = "prefix of URL"+ "c/l?u=10C78AC0&e=17AD89B&c=194D0D&t=0&email=WqS0CM9o%2BpbtiwumbI%2Fj2w%3D%3D&seq=1"
url = 'https://www.example.org/bin.exe?arg=value'
classification = predict_url_classification(url, dTree, df_final_columns)
print(f"Classes: 'benign': 0, 'defacement': 1, 'malware': 2, 'phishing': 3")
print(f"Class of URL: {classification}")

Classes: 'benign': 0, 'defacement': 1, 'malware': 2, 'phishing': 3
Class of URL: 0


In [65]:
url = 'http://clt1658125.benchurl.com/c/l?u=10C78AC0&e=17AD89B&c=194D0D&t=0&email=WqS0CM9o%2BpbtiwumbI%2Fj2w%3D%3D&seq=1'
classification = predict_url_classification(url, dTree, df_final_columns)
print(f"Classes: 'benign': 0, 'defacement': 1, 'malware': 2, 'phishing': 3")
print(f"Class of URL: {classification}")

Classes: 'benign': 0, 'defacement': 1, 'malware': 2, 'phishing': 3
Class of URL: 0


In [67]:
url = 'https://mail.google.com/mail/u/0/#inbox/FMfcgzGxRxCPkgxtFFtqQZmxSzPPwvbX'
classification = predict_url_classification(url, dTree, df_final_columns)
print(f"Class of URL: {classification}")

Class of URL: 0
